# PROJECT COMPONENT 3

### Importing all the important libraries so that we dont need to call it again and again.
### loading the file into UFO which contains information about different fields such date and location

In [1]:
import pandas as pd
from bqplot import Tooltip,interacts
from ipywidgets import interact
import ipywidgets as widgets 
import bqplot
import us


names = ["date", "city", "state", "country", "shape", "duration_seconds",
         "duration_reported", "description", "report_date", "latitude",
         "longitude"]

fn = "/srv/nbgrader/data/ufo-scrubbed-geocoded-time-standardized.csv"
ufo = pd.read_csv(fn, names = names, parse_dates = ["date", "report_date"])

abbr_to_fits = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fits.get(str(a).upper(), -1)))
total_time = ufo.groupby("fips")['duration_seconds'].sum()
total_sightings = ufo.groupby("fips")['report_date'].count()


### Extracting the year so it can be used in a better way and renaming the columns earlier to have a better understanding

In [2]:
import matplotlib.pyplot as plt
ufo['year'] = pd.DatetimeIndex(ufo['date']).year
ufo.columns = ["date", "city", "state", "country", "shape", "total_time",
         "duration_reported", "description", "total_sightings", "latitude",
         "longitude",'fips','year']

In [3]:
df = pd.DataFrame([total_sightings,total_time])
df = df.T
df.columns = ['total_sightings','total_time']
df =df[1:]

### This function updates the scatter plot by using the year value, state id is the fip number and the  field if its either total_time and total_sightings

In [4]:
from bqplot import *

def getPlot(year):  
    
    ufo_state =ufo[ufo['fips'] == state_id]
    ufo_plot = ufo_state[ufo_state['year'] > year[0]]
    
    if(field1 == "total_time"):
        value_counts = ufo_state.groupby(["year"])["total_time"].sum()
    else:
        value_counts = ufo_state.groupby(["year"])["total_sightings"].count()

    wealth_scat.x = sorted(ufo_plot['year'].unique())
    wealth_scat.y = value_counts
    

### This function restricts the interaction on the map to one state only, as we only want a single entity to be selected rather than a list of selected states

In [5]:
def observe_map(change):
    global state_id


    if len(change.new) > 1:
        states_map.selected = [change.new[-1]]
    elif len(change.new) == 1:
    
        state_id = states_map.selected[-1:]
        ufo_state = ufo[ufo['fips'] == state_id[0]]

       
        
        scatter_plot = interact(getPlot, year = widgets.IntRangeSlider(
                    value=[ufo_state['year'].max() - 50,ufo_state['year'].max()],
                    min=ufo_state['year'].min(),
                    max=ufo_state['year'].max(),
                    continuous_update=True,
                    step=10,
               ));
        
    
    

### This function creates the elegant map using bqplot and adds a tooltip while we hover on states and the observe function calls the observe_map and the selected state is passed to it

In [6]:
def make_plot(field):
    global field1
    field1=field
    map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                             'color': bqplot.ColorScale(colors=["orange","red", "black"])},
                  'color': df[field].to_dict(),}
    map_tt = Tooltip( fields = ['color','id','name']  ,labels = [field,'id','name'])
    global states_map
    states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'),**map_styles, interactions={'click': 'select','hover': 'tooltip'},selected_style={'stroke': 'navy'})
    states_map.tooltip = map_tt
    states_map.observe(observe_map, 'selected')
    map_fig = bqplot.Figure(marks=[states_map], title='USA',height = 100, width = 100)
    return map_fig

### This code first creates an empty scatter plot which would be later updated whenevr the states are selected and also it calls the make_plot function so that the map could be plotted and then a series of function are called to make this interatcive

In [7]:
sc_x = LinearScale()
sc_y = LinearScale()
ax_x = Axis(label='Year', scale=sc_x)
ax_y = Axis(label= 'total', scale=sc_y, orientation='vertical', tick_format='0.2f')
wealth_scat = Lines(x=[], y=[], colors=['blue'],
                      display_names=False,
                      scales={'x': sc_x, 'y': sc_y},)
wealth_scat.colors =["orange"]

fig = Figure(marks=[wealth_scat], axes=[ax_x, ax_y],
             title='United States')

%matplotlib inline
map_fig = interact(make_plot,field = widgets.Dropdown(
    options =["total_sightings","total_time"],
    disabled=False,
))
fig

    

interactive(children=(Dropdown(description='field', options=('total_sightings', 'total_time'), value='total_sightings'), Output()), _dom_classes=('widget-interact',))

Figure(axes=[Axis(label='Year', scale=LinearScale()), Axis(label='total', orientation='vertical', scale=LinearScale(), tick_format='0.2f')], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, layout=Layout(min_width='125px'), marks=[Lines(colors=['orange'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9})], scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0), title='United States')

## Approach 

### Learnt from the component 1 mistake used same building block for making the plots. it was easier and compatible to use it. BQplot although does not have much information about it except on its github page which was later found to be useful, rest everything is explained in the cells

## Strength

### Using BQplot for both the elements was a good choice as it does not act weird as they the elements are compatible with each other so even using ipywidgets it did not create problems, also the code is very readable and written by defining functions.

## Weakness

### I would not call this a weakness but a funny story about this, initially i was getting multiple plots for yeartime i click on different state so if i clicked on 2 states i would get 2 plots. after wrecking my head on this for a while(3 days) i came to a point where there was just a single plot and it was updating on that BUT now multiple sliders started coming HAHA. "I guess what goes around comes around "

## What I wish I could do 

### First, normalizaion which we could not due to poor time management
### Second, Try to figure out how to remove multiple slider( we did think on it alot but got nothing , nothing !!)
### Third, a slight obervation in total time, even though it shows arkansas as the states with highest total time, there is a outlier point in the data and as we are taking sum it is will take it to the larger side of the spectrum. DO check it when you grade this, i couldnt show it much due to time constrains

## End Remarks

### Thank you to Prof Matthew Turk , TA Jerai Carter , friends, google and stackoverflow for all the help that youl provide to the almost successful completion of the project.
### To compensate for not completing some features, do check our infographics
### IT is said that the effort counts, haha :P